In [ ]:
import arcpy
from arcpy import env
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import pysal as ps
import glob
import shutil
import pathlib
from IPython.display import display

In [ ]:
tempDir = "C:\TempAnalysis"
dataDir = "C:\TestData"

In [ ]:
def buildTempWorkspace(tempDir):
    if not os.path.exists(tempDir):
        os.mkdir(tempDir)
        print("Directory " ,tempDir,  " Created.")
    else:    
        print("Directory " ,tempDir,  " already exists.")

In [ ]:
def buildAnalysisDatabaseList(dataDir):
    env.workspace = dataDir
    fgdbList = arcpy.ListWorkspaces("*", "FileGDB")
    return fgdbList

In [ ]:
def buildAnalysisTables(fgdb,tempDir):
    env.workspace = fgdb        
    fcpList = []
    dsList = arcpy.ListDatasets(feature_type='feature')
    dsList = [''] + dsList if dsList is not None else []
    for ds in dsList:
        for fc in arcpy.ListFeatureClasses(feature_dataset=ds):
            path = os.path.join(fgdb, ds, fc)
            fcpList.append(path)
    fgdbDirName = os.path.splitext(os.path.basename(fgdb))[0]
    fgdbDirPath = os.path.join(tempDir,fgdbDirName)
    if not os.path.exists(fgdbDirPath):
        os.mkdir(fgdbDirPath)
        print("Directory " ,fgdbDirPath,  " created. ")
    else:    
        print("Directory " ,fgdbDirPath,  " already exists.")
    for fcp in fcpList:
        print(fcp)
        arcpy.TableToTable_conversion(fcp, fgdbDirPath,os.path.splitext(os.path.basename(fcp))[0])

In [ ]:
def buildDBFPList(fgdb):
    dbfpList = []
    fgdbDirName = os.path.splitext(os.path.basename(fgdb))[0]
    fgdbDirPath = os.path.join(tempDir,fgdbDirName)
    for p in pathlib.Path(fgdbDirPath).glob('*.dbf'):
        if p.is_file():
            dbfpList.append(str(p))
    return(dbfpList)

In [ ]:
def dbf2DF(dbfp, upper=True):
    db = ps.open(dbfp)
    d = {col: db.by_col(col) for col in db.header}
    pandasDF = pd.DataFrame(d)
    if upper == True:
        pandasDF.columns = map(str.upper, db.header) 
    db.close()
    return pandasDF

In [ ]:
def dbfIterate(dbfpList):
    for dbfp in dbfpList:
        print(os.path.splitext(os.path.basename(dbfp))[0])
        thisDF = (dbf2DF(dbfp, upper=True))
        cleanThisDF = thisDF.dropna()
        display(cleanThisDF)

In [ ]:
def fgdbIterate(fgdbList):
    print("Found the following FGDBs: ",fgdbList)
    for fgdb in fgdbList:
        buildAnalysisTables(fgdb,tempDir)
        dbfIterate(buildDBFPList(fgdb))

In [ ]:
def cleanupTempWorkspace(tempDir):
    if not os.path.exists(tempDir):
        print("Directory " ,tempDir,  " not present.")
    else:
        shutil.rmtree(tempDir)
        print("Directory " ,tempDir,  " cleaned up.")

In [ ]:
buildTempWorkspace(tempDir)
buildAnalysisDatabaseList(dataDir)
fgdbIterate(buildAnalysisDatabaseList(dataDir))
cleanupTempWorkspace(tempDir)